# Chapter 13: Loading and Preprocessing Data with TensorFlow

Notebook ini merupakan hasil reproduksi dan penjelasan teori dari **Bab 13 - Loading and Preprocessing Data with TensorFlow** dari buku *Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow (2nd Edition)* oleh Aurélien Géron.

📌 Bab ini menjelaskan cara menangani data dalam skala besar dan pipeline preprocessing efisien menggunakan `tf.data`, serta penggunaan `TFRecord` untuk format penyimpanan data cepat.

---


## Ringkasan Teori Bab 13: Loading and Preprocessing Data with TensorFlow

### 1. Mengapa Gunakan `tf.data`?

- Menyediakan **pipeline data input yang efisien**
- Kompatibel dengan GPU dan multi-threading
- Bisa menangani data besar dan kompleks (gambar, teks, dll)

---

### 2. Membuat Dataset

```python
dataset = tf.data.Dataset.from_tensor_slices(data_array)
```

Dataset bisa berasal dari:
- List atau array NumPy
- File CSV/TFRecord
- File gambar (image folder)

---

### 3. Transformasi Dataset

Fungsi utama `tf.data.Dataset`:
- `.map(fn)`: memproses setiap elemen dengan `fn`
- `.batch(batch_size)`: menggabungkan dalam batch
- `.shuffle(buffer_size)`: mengacak urutan data
- `.repeat()`: mengulangi dataset

Contoh:
```python
dataset = dataset.shuffle(1000).map(preprocess_fn).batch(32).repeat()
```

---

### 4. Pipeline untuk Model Training

```python
model.fit(dataset, epochs=5, steps_per_epoch=100)
```

Pastikan dataset dibatch dan di-repeat jika digunakan untuk `.fit()`

---

### 5. TFRecord

Format biner yang efisien untuk menyimpan dataset besar.

Keuntungan:
- Dibaca lebih cepat oleh TensorFlow dibanding CSV
- Kompatibel dengan pipeline `tf.data`

---

### 6. Membuat TFRecord

```python
with tf.io.TFRecordWriter(filename) as writer:
    for sample in data:
        feature = {
            'feature_name': tf.train.Feature(...)
        }
        example = tf.train.Example(features=tf.train.Features(feature=feature))
        writer.write(example.SerializeToString())
```

---

### 7. Membaca TFRecord

```python
raw_dataset = tf.data.TFRecordDataset([filename])
parsed_dataset = raw_dataset.map(parse_example_fn)
```

---

### 8. Preprocessing Khusus

Bisa digunakan untuk:
- Normalisasi numerik
- Augmentasi gambar
- Tokenisasi teks

Gunakan `.map()` dan kombinasikan dengan fungsi preprocessing sendiri.

---

### 9. Prefetching dan Parallelism

Pipeline dapat dioptimasi dengan:

```python
dataset = dataset.prefetch(tf.data.AUTOTUNE)
```

Ini memungkinkan eksekusi paralel antara input data dan training model.

---

📌 Dengan `tf.data`, kita bisa membangun pipeline data training yang scalable dan efisien, bahkan untuk proyek deep learning besar di cloud.

---


## Implementasi tf.data Pipeline dan Simulasi TFRecord

Kita akan membuat:
1. Dataset sederhana menggunakan `tf.data.Dataset`
2. Menambahkan transformasi (`shuffle`, `map`, `batch`)
3. Simulasi penulisan dan pembacaan **TFRecord**

---


In [ ]:
import tensorflow as tf

# Dataset numerik sederhana
features = tf.range(10)
labels = tf.range(10) * 2

dataset = tf.data.Dataset.from_tensor_slices((features, labels))
dataset = dataset.shuffle(buffer_size=10).map(lambda x, y: (x, y)).batch(2).prefetch(tf.data.AUTOTUNE)

for batch in dataset.take(3):
    print(batch)


### Penjelasan:
- Dataset terdiri dari pasangan (x, 2x)
- Kita shuffle, lalu batch 2 elemen
- `prefetch` untuk meningkatkan performa saat training


In [ ]:
# Simulasi penulisan TFRecord (ke file sementara)
def create_example(x, y):
    feature = {
        "x": tf.train.Feature(int64_list=tf.train.Int64List(value=[x])),
        "y": tf.train.Feature(int64_list=tf.train.Int64List(value=[y])),
    }
    return tf.train.Example(features=tf.train.Features(feature=feature))

filename = "/mnt/data/sample.tfrecord"
with tf.io.TFRecordWriter(filename) as writer:
    for x_val, y_val in zip(range(5), range(5, 10)):
        example = create_example(x_val, y_val)
        writer.write(example.SerializeToString())

In [ ]:
# Membaca kembali TFRecord
feature_description = {
    "x": tf.io.FixedLenFeature([], tf.int64),
    "y": tf.io.FixedLenFeature([], tf.int64),
}

def parse_example(serialized_example):
    return tf.io.parse_single_example(serialized_example, feature_description)

tfrecord_dataset = tf.data.TFRecordDataset([filename])
tfrecord_dataset = tfrecord_dataset.map(parse_example)

for parsed in tfrecord_dataset:
    print(parsed)


### Penjelasan:
- Kita membuat dan menyimpan 5 contoh pasangan angka (x, y) ke file `.tfrecord`
- Kemudian kita baca kembali menggunakan `TFRecordDataset`
- `parse_example` diperlukan untuk mengubah serialized format ke dictionary Python

📌 Teknik ini digunakan dalam proyek skala besar seperti pengolahan gambar dalam format TFRecord yang dioptimalkan untuk I/O GPU.

---
